# Character-level Language Model
### (CS231n: lecutre 8 - 44)

In [34]:
import torch

vocab = ["h", "e", "l", "o"]
sequence = "hello"

# tokenization(character-level)
seq = list(sequence)                            # 문자열을 개별 문자로 분리

# one-hot encoding
seq_encoding = []
for i in range(len(seq)):
    encoding = [0] * len(vocab)                 # 리스트 내부를 모두 0으로 채우기
    for index, value in enumerate(vocab):       # 인덱스와 인덱스에 해당하는 리스트 요소 가져오기
        if(seq[i] == value):
            encoding[index] = 1
            seq_encoding.append(encoding)

seq_encoding = torch.tensor(seq_encoding)       # 리스트를 tensor로 변환
print(seq_encoding)
print(seq_encoding.shape)


tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
torch.Size([5, 4])


In [42]:
import numpy as np

# weight matrix 초기화
W_hh = np.random.randn(3, 3)            # 크기가 3x3인 난수 행렬 생성 (정규분포)
W_hh = torch.from_numpy(W_hh)           # numpy를 tensor로 변환
print(W_hh.shape)

W_xh = np.random.randn(4, 3)
W_xh = torch.from_numpy(W_xh)
print(W_xh.shape)

W_hy = np.random.randn(3, 4)
W_hy = torch.from_numpy(W_hy)
print(W_hy.shape)

# 초기 hidden state 값을 0으로 설정
h_0 = torch.zeros(3, 3)                 # tensor([[0., 0., 0.],
                                              #   [0., 0., 0.],
                                              #   [0., 0., 0.]])
print(h_0.shape)

torch.Size([3, 3])
torch.Size([4, 3])
torch.Size([3, 4])
torch.Size([3, 3])


In [43]:
# <주의>
# * 연산자는 요소별 곱셈을 수행함
# torch.matmul() or @ 연산자가 행렬 곱셈을 수행함

hidden_state = torch.matmul(W_hh, h_0)

input = torch.matmul(W_xh, seq_encoding[0])


hidden_state

RuntimeError: expected m1 and m2 to have the same dtype, but got: double != float